In [6]:
from ngsolve import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.krylovspace import CGSolver
from ngsolve.webgui import Draw
from netgen.csg import unit_cube

def Div(q):
    if q.dim == 3:
        return CF(q[0].Diff(x) + q[1].Diff(y) + q[2].Diff(z) )
    if q.dim == 9:
        return CF( (Div(q[0,:]),Div(q[1,:]),Div(q[2,:])),dims=(3,1) )

def Grad(q):
    if q.dim == 1:
        return CF((q.Diff(x),q.Diff(y),q.Diff(z)) )
    if q.dim == 3:
        return CF( (Grad(q[0]),Grad(q[1]),Grad(q[2])),dims=(3,3) )


n = specialcf.normal(3)
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)
P_n = OuterProduct(n,n) 
Q_n = Id(3) - OuterProduct(n,n) 
C_n = CF( (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) )
def C(V): 
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 
def L2Inc(gfG, Mesh, Order):
    mesh = Mesh
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():

        gfCurlCurlG = GridFunction(fesHCurlCurl, order=Order,dirichlet= ".*")
        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl)
        a += InnerProduct(u,v)*dx 

        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        
        a.Assemble()
        f.Assemble()

        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        #inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=500,tol=1e-9)
        gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
        #gfCurlCurlG.vec.data += inverse * r

        return gfCurlCurlG

def BeltramiDecomposition(F, mesh, mu=1, order=3, symgrad_phi=None, inc_A=None, draw=False):
    if order < 2:
        raise Exception("Wrong order!")

    ##########################
    #   determine inc part  #
    ##########################
    
    # the bilinear form (and the linear one) we consider is the one that comes 
    # from the minimization problem : || inc(u)- F||^2 : u * Grad(p) = 0 for all p in H1
    
    fes = HCurlCurl(mesh, order=order, dirichlet=".*")*VectorH1(mesh,order=order+1, dirichlet=".*")
    (u,p), (v,q) = fes.TnT()
    print("a")
    a = BilinearForm(fes, symmetric=True, symmetric_storage=True)#, condense=True)
    print("b")
    a += mu*InnerProduct(u.Operator("inc"),v.Operator("inc"))*dx
    print("c")
    a += InnerProduct(p,Div(v))*dx
    print("d")
    a += InnerProduct(q,Div(u))*dx
    print("e")

    f = LinearForm(fes)
    print("f")
    f += InnerProduct(F,v.Operator("inc"))*dx
    #f += InnerProduct(F,v)*dx
    print("g")

    a.Assemble()
    print("h")
    f.Assemble()
    print("i")
    gfsol = GridFunction(fes)
    print("j")
    r = f.vec.CreateVector()
    print("k")
    w = f.vec.CreateVector()
    print("l")
    
    inv = a.mat.Inverse(fes.FreeDofs(), inverse="pardiso")
    print("m")
                
    r.data = f.vec
    print("n")
    if a.condense:
        print("o")
        r.data += a.harmonic_extension_trans * r
    print("p")

    w.data = inv * r
    print("q")
    if a.condense:
        print("r")
        w.data += a.harmonic_extension * w
        print("s")
        w.data += a.inner_solve * r
        print("t")
    gfsol.vec.data = w
    print("u")
                    
    gfA, gfp = gfsol.components
    print("v")

    if True:
        print("gfA")
        Draw(gfA, mesh, "gfA",clipping=(0,0,1))
        print("w")

    # to create inc_gfA we need to use a solver for a "direct problem"  
    #inc_gfA = IN
    inc_gfA = L2Inc(gfA,mesh,order)
    print("x")
    if draw:
        print("inc_gfA")
        Draw(BoundaryFromVolumeCF(inc_gfA), mesh, "inc_gfA",clipping=(0,0,1))
        print("y")

    input("determine the sumgrad part")
    # determine grad part
    numberspace3 = NumberSpace(mesh)*NumberSpace(mesh)*NumberSpace(mesh)
    fes2 = VectorH1(mesh,order=order)*numberspace3
    (u,alpha1,alpha2,alpha3),(v,beta1,beta2,beta3) = fes2.TnT()
    a2 = BilinearForm(fes2, symmetric=True, symmetric_storage=True)
    a2 += (InnerProduct(Sym(Grad(u)),Sym(Grad(v))) + alpha1*v[0]+ alpha2*v[1] + alpha3*v[2]+ beta1*u[0]+beta2*u[1]+beta3*u[2])*dx

    f2 = LinearForm(fes2)
    f2 += InnerProduct(F,Sym(Grad(v)))*dx

    a2.Assemble()
    f2.Assemble()

    gfsol2 = GridFunction(fes2)
    gfphi, gfN1,gfN2 ,gfN3= gfsol2.components

    gfsol2.vec.data = a2.mat.Inverse(fes2.FreeDofs(), inverse="pardiso")*f2.vec

    if draw:
        print("gfphi")
        Draw(gfphi, mesh, "gfphi",clipping=(0,0,1))
        print("symgrad_gfphi")
        Draw(BoundaryFromVolumeCF(Sym(Grad(gfphi))), mesh, "symgrad_gfphi",clipping=(0,0,1))

    print(" err F: ", sqrt(Integrate(InnerProduct(Sym(Grad(gfphi))+inc_gfA-F,Sym(Grad(gfphi))+inc_gfA-F)**2,mesh)))
    if symgrad_phi:
        print("err grad_phi: ", sqrt(Integrate(InnerProduct(Sym(Grad(gfphi))-symgrad_phi,Sym(Grad(gfphi))-symgrad_phi),mesh)))
    if inc_A:
        print("err curl_A: ", sqrt(Integrate(InnerProduct(inc_gfA-inc_A,inc_gfA-inc_A),mesh)))

        
    return gfA, gfphi, inc_gfA

#mesh = MakeStructured3DMesh(False, nx=4,ny=4,nz=4, mapping = lambda x,y,z :(2*x-1,2*y-1,2*z-1))
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.3))
draw = False

phi =CF( ( 10**5*((1-x)*x)**2*((1-y)*y)**2*((1-z)*z)**2 , 0 , 0) )
if draw :
    print("phi")
    Draw(phi,mesh,clipping=(0,0,1))

symgrad_phi = CF( Sym(Grad(phi)) )
if draw :
    print("symgrad_phi")
    Draw(symgrad_phi,mesh,clipping=(0,0,1))

peak = ( 1-(x-0.5)**2 - (y-0.5)**2 )**2
A =   CF ( (0, 0, 0 , 0,0,0 ,0,0,peak), dims=(3,3))
if draw :
    print("A")
    Draw(InnerProduct(A,A),mesh,clipping=(0,0,1))

if draw :
    print("div_A")
    Draw(InnerProduct(Div(A),Div(A)),mesh,clipping=(0,0,1))

inc_A = CF( Inc(A) )
if draw :
    print("inc_A")
    Draw(InnerProduct(inc_A,inc_A),mesh,clipping=(0,0,1))
    for i in [0,1,2]:
        for j in [0,1,2]:
            print("inc_A("+str(i) +str(j)+") - " + " A("+str(i) +str(j)+") -")
            Draw(inc_A[i,j]-A[i,j],mesh,clipping=(0,0,1))

F = symgrad_phi+inc_A
if draw :
    print("F")
    Draw(InnerProduct(F,F),mesh,clipping=(0,0,1))


with TaskManager():
    gfA, gfphi, inc_gfA = BeltramiDecomposition(F, mesh,mu=0, order=2, symgrad_phi=symgrad_phi, inc_A=inc_A, draw=True)
print(" error gfA - A " + str(sqrt(Integrate(InnerProduct(A-gfA,A-gfA),mesh))))

a
b
c
d
e
f
g
h
i
j
k
l
m
n
p
q
u
v
gfA


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

w
x
inc_gfA


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

y
gfphi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

symgrad_gfphi


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

 err F:  3.7956477485850723e+39
err grad_phi:  30.981480688692113
err curl_A:  4.104369732813052e+19
 error gfA - A 7.237519198379722e+16


In [7]:
help(Grad(CF((x,y,z))))

Help on CoefficientFunction in module ngsolve.fem object:

class CoefficientFunction(pybind11_builtins.pybind11_object)
 |  A CoefficientFunction (CF) is some function defined on a mesh.
 |  Examples are coordinates x, y, z, domain-wise constants, solution-fields, ...
 |  CFs can be combined by mathematical operations (+,-,sin(), ...) to form new CFs
 |  Parameters:
 |  
 |  val : can be one of the following:
 |  
 |    scalar (float or complex):
 |      Creates a constant CoefficientFunction with value val
 |  
 |    tuple of scalars or CoefficientFunctions:
 |      Creates a vector or matrix valued CoefficientFunction, use dims=(h,w)
 |      for matrix valued CF
 |    list of scalars or CoefficientFunctions:
 |      Creates a domain-wise CF, use with generator expressions and mesh.GetMaterials()
 |      and mesh.GetBoundaries()
 |  
 |  Method resolution order:
 |      CoefficientFunction
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here: